Test of library Requests with boursorama 

In [2]:
import requests
import pandas as pd
url = f"https://www.boursorama.com/bourse/trackers/cours/historique/1rTWPEA"

response = requests.get(url, timeout=5)  # 5 secondes max
df_list = pd.read_html(response.text)
pd.DataFrame(df_list[0])

,Date,Dernier,Var. %,+ haut,+ bas,Ouverture
0,27/05/2025,547,"+0,76%",548,543,544
1,26/05/2025,543,"+0,90%",544,533,543
2,23/05/2025,538,"-1,03%",544,533,543
3,22/05/2025,544,"-0,75%",545,540,543
4,21/05/2025,548,"-0,83%",549,544,548
5,20/05/2025,553,"+0,23%",554,551,552
6,19/05/2025,551,"-0,56%",552,545,549
7,16/05/2025,554,"+0,75%",554,551,551
8,15/05/2025,550,"+0,38%",551,545,546
9,14/05/2025,548,"-0,31%",550,545,549


il faut voir un moyen de passer de l'ISIN au ticker
Par défaut, on a un mois d'histo. Pour avoir plus, il faut:
- choisir 3Y 
- appuyer sur rechercher 
- appuyer sur le numero de la page + 1

Request nous permet de lire l'html mais pas faire du dynamique. Pour simuler du click bouton, il faut utiliser Selenium. 


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

url="https://www.boursorama.com/bourse/trackers/cours/historique/1rTWPEA"
driver = webdriver.Chrome()

# Ouvre la page
driver.get(url) 
# Pause pour observer 
time.sleep(2)
driver.quit()

Problème identifié: pop up des cookies à gérer

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import time

url="https://www.boursorama.com/bourse/trackers/cours/historique/1rTWPEA"
driver = webdriver.Chrome()

# Ouvre la page
driver.get(url)  # Remplace par ton URL
wait = WebDriverWait(driver, 10)

try:
    continue_button = wait.until(EC.element_to_be_clickable(
        (By.CLASS_NAME, "didomi-continue-without-agreeing")
    ))
    continue_button.click()
    print("Bouton 'Continuer sans accepter' cliqué.")
except Exception as e:
    print("Bouton cookies pas trouvé ou déjà accepté :", e)

time.sleep(5)

driver.quit()

Bouton 'Continuer sans accepter' cliqué.


In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import time

url="https://www.boursorama.com/bourse/trackers/cours/historique/1rTWPEA"
driver = webdriver.Chrome()

# Ouvre la page
driver.get(url)  # Remplace par ton URL
wait = WebDriverWait(driver, 10)

try:
    continue_button = wait.until(EC.element_to_be_clickable(
        (By.CLASS_NAME, "didomi-continue-without-agreeing")
    ))
    continue_button.click()
    print("Bouton 'Continuer sans accepter' cliqué.")
except Exception as e:
    print("Bouton cookies pas trouvé ou déjà accepté :", e)

time.sleep(5)

# Sélectionner "3 ans"
select_element = wait.until(EC.visibility_of_element_located((By.ID, "historic_search_duration")))
select = Select(select_element)
select.select_by_value("3Y")

time.sleep(5)

driver.quit()

Bouton 'Continuer sans accepter' cliqué.


TimeoutException: Message: 


Je sens que cette piste va me prendre beaucoup de temps, donc je vais chercher une API.
Recours à l'API https://marketstack.com/dashboard 

In [3]:
from dotenv import load_dotenv
import os
load_dotenv()
API_KEY=os.getenv("API_key_market_stack")

In [6]:
import requests

url = f"https://api.marketstack.com/v1/eod?access_key={API_KEY}"

querystring = {"symbols":"AAPL","date_from":"2025-05-20" ,"date_to":"2025-05-27"}

response = requests.get(url, params=querystring)

print(response.json())

{'pagination': {'limit': 100, 'offset': 0, 'count': 5, 'total': 5}, 'data': [{'open': 198.3, 'high': 200.74, 'low': 197.43, 'close': 200.21, 'volume': 56229000.0, 'adj_high': 200.74, 'adj_low': 197.43, 'adj_close': 200.21, 'adj_open': 198.3, 'adj_volume': 56288475.0, 'split_factor': 1.0, 'dividend': 0.0, 'symbol': 'AAPL', 'exchange': 'XNAS', 'date': '2025-05-27T00:00:00+0000'}, {'open': 193.665, 'high': 197.7, 'low': 193.46, 'close': 195.27, 'volume': 78432918.0, 'adj_high': 197.7, 'adj_low': 193.46, 'adj_close': 195.27, 'adj_open': 193.665, 'adj_volume': 78432918.0, 'split_factor': 1.0, 'dividend': 0.0, 'symbol': 'AAPL', 'exchange': 'XNAS', 'date': '2025-05-23T00:00:00+0000'}, {'open': 200.71, 'high': 202.75, 'low': 199.7, 'close': 201.36, 'volume': 46633900.0, 'adj_high': 202.75, 'adj_low': 199.7, 'adj_close': 201.36, 'adj_open': 200.71, 'adj_volume': 46742407.0, 'split_factor': 1.0, 'dividend': 0.0, 'symbol': 'AAPL', 'exchange': 'XNAS', 'date': '2025-05-22T00:00:00+0000'}, {'open': 

In [8]:
json_response=response.json()

In [12]:
import pandas as pd
pd.DataFrame(json_response['data'])

,open,high,low,close,volume,adj_high,adj_low,adj_close,adj_open,adj_volume,split_factor,dividend,symbol,exchange,date
0,198.300,200.74,197.43,200.21,56229000.0,200.74,197.43,200.21,198.300,56288475.0,1.0,0.0,AAPL,XNAS,2025-05-27T00:00:00+0000
1,193.665,197.70,193.46,195.27,78432918.0,197.70,193.46,195.27,193.665,78432918.0,1.0,0.0,AAPL,XNAS,2025-05-23T00:00:00+0000
2,200.710,202.75,199.70,201.36,46633900.0,202.75,199.70,201.36,200.710,46742407.0,1.0,0.0,AAPL,XNAS,2025-05-22T00:00:00+0000
3,205.170,207.04,200.71,202.09,59134800.0,207.04,200.71,202.09,205.170,59211774.0,1.0,0.0,AAPL,XNAS,2025-05-21T00:00:00+0000
4,207.670,208.47,205.03,206.86,42443700.0,208.47,205.03,206.86,207.670,42496635.0,1.0,0.0,AAPL,XNAS,2025-05-20T00:00:00+0000
